In [1]:
import pandas as pd
import random

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
INPUT_FILE = '../toxicity_ratings - twitter.csv'

In [4]:
df = pd.read_csv(INPUT_FILE)

/Users/shagunjhaver/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,9,10,14,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df = df.dropna()

In [6]:
def get_avg_toxicity(row):
    result = (row['toxic_score'] + row['toxic_score.1'] + row['toxic_score.2'] + row['toxic_score.3'] + row['toxic_score.4'])/20.0
    return result

In [7]:
df['avg_toxic_score'] = df.apply (lambda row: get_avg_toxicity(row), axis=1)

In [8]:
df = df[['comment', 'perspective_score', 'avg_toxic_score', 'comment_ID']]

In [9]:
def get_bucket(row, field):
    if (row[field] < 0.2):
        return 0.0
    elif (row[field] < 0.4):
        return 0.2
    elif (row[field] < 0.6):
        return 0.4
    elif (row[field] < 0.8):
        return 0.6
    else:
        return 0.8

In [10]:
df['perspective_window'] = df.apply(lambda row: get_bucket(row, 'perspective_score'), axis=1)

In [11]:
df['avg_toxic_window'] = df.apply(lambda row: get_bucket(row, 'avg_toxic_score'), axis=1)


In [12]:
f = df[df['perspective_window'] != df['avg_toxic_window']]

In [13]:
len(f)

58321

In [14]:
myrange = [0.0, 0.2, 0.4, 0.6, 0.8]
sampled_dfs = []
for ow in myrange:
    g = f[f['perspective_window'] == ow]
    for iw in myrange:
        if (ow != iw):
            h = g[g['avg_toxic_window'] == iw]
            sampled_dfs.append(h.sample(1))

In [15]:
usernameDf = pd.read_csv('../usernames.csv')
usernames = set(list(usernameDf['username']))

FileNotFoundError: [Errno 2] No such file or directory: 'usernames.csv'

In [ ]:
final_df = pd.concat(sampled_dfs)

In [ ]:
final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
final_df

In [ ]:
final_df.to_csv('main_comments.csv', columns = ['comment', 'author', 'perspective_score', 'avg_toxic_score', 'comment_ID'], index=False)